# Geology Extraction from the Shapefiles

I have extracted a subset of shapefiles for the geologic infromation in my area of interest (AOI), from a larger geologic map of the United States. I am testing for the interection of a `latitude` and `longitude` point in one of the shapefiles for the different lithological units in the AOI. If the points interest, I am then extracting the geologic information from the lithologic unit and adding it the existing `.csv` files. This lithological information will then be used as predictors for the location and magntiude of anthropogenically induced earthquakes.  

The spatial indexing in incredible computationaly expensive, based on the fact that I have a shapefile with $\sim$90,000 polygons which need to check if they interect with $\sim$ 15,000 recorded earthquakes. For this reason I am using a spatial indexing method, to increase the efficency of the computation. Explaination of the R-tree spatial indexing and examples can be founf [here](https://geoffboeing.com/2016/10/r-tree-spatial-index-python/). From a test subset of 2000 polygons and 50 focal points, we see a five fold incease in computation time between brute force and the spatial index method. More examples of R-tree spatial indexing can be found [here](https://stackoverflow.com/questions/55722066/python-shapely-aggregating-points-to-shape-files-for-a-choropleth-map), and [here](https://stackoverflow.com/questions/37934023/how-to-use-geopandas-spatial-index-with-lines).

In [1]:
%matplotlib notebook
import shapely
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

Read in the Geologic Map of the US, clipped to our AOI. 

In [2]:
%time AOI_shp = gpd.read_file('../data/shp/AOI_Geology.shp')

CPU times: user 17.4 s, sys: 770 ms, total: 18.2 s
Wall time: 18.2 s


Read in the earth quake csv and convert lat, lon points to "shapepoints" via `geopandas`

In [70]:
ok_quakes = pd.read_csv('../data/okQuakes.csv')

ok_quakes['geometry'] = list(zip(ok_quakes.longitude, ok_quakes.latitude))
ok_quakes['geometry'] = ok_quakes['geometry'].apply(shapely.geometry.Point)

quakes_shp = gpd.GeoDataFrame(ok_quakes, geometry='geometry', crs = {'init': 'epsg:4326'})

Read in the injection well csv and convert lat, lon points to "shapepoints" via `geopandas`, and create a subset of the Geologic Map which just includes Oklahoma since all of the injection well site as within the state 

In [71]:
inject_wells = pd.read_csv('../data/InjectionWells.csv')

inject_wells['geometry'] = list(zip(inject_wells.LONG, inject_wells.LAT))
inject_wells['geometry'] = inject_wells['geometry'].apply(shapely.geometry.Point)

inject_wells_shp = gpd.GeoDataFrame(inject_wells, geometry='geometry', crs = {'init': 'epsg:4326'})

OK_geol_shp = AOI_shp.loc[AOI_shp.STATE == 'OK']

Spatial Indexing function 

In [47]:
def spatial_index(quakes_df, geology_df):
    spatial_index = quakes_df.sindex
    for i, unit in geology_df.iterrows():
        polygon = unit.geometry
        possible_matches_index = list(spatial_index.intersection(polygon.bounds))
        possible_matches = quakes_df.iloc[possible_matches_index]
        precise_matches = possible_matches[possible_matches.within(polygon)]
        if precise_matches.empty != True:
            quakes_df.at[precise_matches.index, 'STATE'] =  unit.STATE
            quakes_df.at[precise_matches.index, 'UNIT_NAME'] =  unit.UNIT_NAME
            quakes_df.at[precise_matches.index, 'AGE_MIN'] =  unit.AGE_MIN
            quakes_df.at[precise_matches.index, 'AGE_MAX'] =  unit.AGE_MIN
            quakes_df.at[precise_matches.index, 'MAJOR1'] =  unit.MAJOR1
            quakes_df.at[precise_matches.index, 'MAJOR2'] =  unit.MAJOR2
            quakes_df.at[precise_matches.index, 'MAJOR3'] =  unit.MAJOR3
            quakes_df.at[precise_matches.index, 'GENERALIZE'] =  unit.GENERALIZE     
    return quakes_df

In [50]:
%%time 
quakes_w_geol_shp = spatial_index(quakes_shp, AOI_shp)

CPU times: user 8min 14s, sys: 3 s, total: 8min 17s
Wall time: 8min 19s


In [60]:
okQuakes_w_geology = quakes_w_geol_shp.loc[:,quakes_w_geol_shp.columns != 'geometry']
okQuakes_w_geology.to_csv('../data/okQuakes_w_geology.csv')

In [72]:
%%time 
inject_wells_w_geol_shp = spatial_index(inject_wells_shp, OK_geol_shp)

CPU times: user 2min 32s, sys: 1.18 s, total: 2min 33s
Wall time: 2min 35s


In [73]:
InjectionWells_w_geology = quakes_w_geol_shp.loc[:,quakes_w_geol_shp.columns != 'geometry']
InjectionWells_w_geology.to_csv('../data/InjectionWells_w_geology.csv')